# End of week 2 Exercise - Bookstore Assistant

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
    
MODEL_GPT = "gpt-4.1-mini"
MODEL_GEMINI = "gemini-2.5-pro"


openai = OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyCL


In [ ]:
# Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models

system_message= """
    You are an assistant in a software engineering bookstore that analyzes the content of technical books and generates concise, informative summaries for readers.
    Your goal is to help customers quickly understand what each book covers, its practical value, and who would benefit most from reading it.
    Respond in markdown without code blocks.
    Each summary should include:
    Overview: The book’s main topic, scope, and focus area (e.g., software architecture, DevOps, system design).
    Key Insights: The most important lessons, principles, or methodologies discussed.
    Recommended For: The type of reader who would benefit most (e.g., junior developers, engineering managers, backend specialists).
    Related Reads: Suggest one or two similar or complementary titles available in the store.
    Maintain a professional and knowledgeable tone that reflects expertise in software engineering literature. 
"""

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model=MODEL_GEMINI,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result


message_input = gr.Textbox(label="Your message:", info="Enter a software engineering book title for the LLM", lines=4)
model_selector = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="Bookstore Assistant", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain Clean Code by Robert C. Martin", "GPT"],
            ["Explain Clean Code by Robert C. Martin", "Gemini"]
        ], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
import sqlite3

DB = "books.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (title TEXT PRIMARY KEY, price REAL)')
    conn.commit()

def get_book_price(title):
    print(f"DATABASE TOOL CALLED: Getting price for {title}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE title = ?', (title.lower(),))
        result = cursor.fetchone()
        return f"Book -> {title} price is ${result[0]}" if result else "No price data available for this title"

def set_book_price(title, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (title, price) VALUES (?, ?) ON CONFLICT(title) DO UPDATE SET price = ?', (title.lower(), price, price))
        conn.commit()

book_prices = {"Clean code":20, "Clean architecture": 30, "System design": 40, "Design patterns": 50}
for title, price in book_prices.items():
    set_book_price(title, price)

In [ ]:
# use of a tool
MODEL = "gpt-4.1-mini"

system_message = """
You are a helpful assistant in a software engineering bookstore BookEye. 
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

price_function = {
    "name": "get_book_price",
    "description": "Get the price of a book.",
    "parameters": {
        "type": "object",
        "properties": {
            "book_title": {
                "type": "string",
                "description": "The title of the book that the customer wants to buy",
            },
        },
        "required": ["book_title"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": price_function}]


def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="coral",
      input=message
    )
    return response.content

def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_book_price":
            arguments = json.loads(tool_call.function.arguments)
            title = arguments.get('book_title')
            price_details = get_book_price(title)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)
    
    return history, voice

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=300, type="messages")
        audio_output = gr.Audio(autoplay=True)
        
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output]
    )

ui.launch(inbrowser=True, auth=("ted", "mowsb"))